# 错误处理


In [2]:
try:
    print('try...')
    r = 10 / int('2')
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as e:
    print('ZeroDivisionError:', e)
else:
    print('no error!')
finally:
    print('finally...')
print('END')

try...
result: 5.0
no error!
finally...
END


In [3]:
import logging

def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        logging.exception(e)

main()
print('END')

ERROR:root:division by zero
Traceback (most recent call last):
  File "<ipython-input-3-cc27997fe30e>", line 11, in main
    bar('0')
  File "<ipython-input-3-cc27997fe30e>", line 7, in bar
    return foo(s) * 2
  File "<ipython-input-3-cc27997fe30e>", line 4, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero


END


## 抛出错误


In [4]:
class FooError(ValueError):
    pass

def foo(s):
    n = int(s)
    if n==0:
        raise FooError('invalid value: %s' % s)
    return 10 / n

foo('0')

FooError: invalid value: 0

# OOC
## map
把一个iterable的对象，每一个元素都用func处理一遍，形成一个映射

In [1]:
#map
def normalize(name):
    return name.lower()
# 测试:
L1 = ['adam', 'LISA', 'barT']
L2 = list(map(normalize, L1))
print(L2)

L3= list(map(lambda x:x.lower(), L1))
print(L3)

['adam', 'lisa', 'bart']
['adam', 'lisa', 'bart']


## reduce
把一个iterable的对象，两两处理，输出一个最终值，可以加入初始值

In [17]:
#reduce
from functools import reduce
a=(reduce(lambda x,y: x*10+y, [1,2,3,4],1))
print(a)

11234


## filter
以指定条件过滤

In [19]:
# filter
a = list(filter(lambda x:x%2 ==0, [i for i in range(10)]))
a

[0, 2, 4, 6, 8]

In [26]:
# filter+ generator
def gen():
    i = 0
    while True:
        yield i
        i += 1
num = gen()
print(next(filter(lambda x:x%10 ==0, num)))
print(next(filter(lambda x:x%10 ==0, num)))
print(next(filter(lambda x:x%10 ==0, num)))

0
10
20


In [27]:
num = filter(lambda x:x%10 ==0, num)
num = filter(lambda x:x%9 ==0, num)
next(num)

90

## 装饰器
这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。
本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下：

In [28]:
def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

@log
def now():
    import time
    print(time.ctime())
now()

call now():
Sun Aug 11 12:32:11 2019


把@log放到now()函数的定义处，相当于执行了语句：
`now = log(now)`  
由于log()是一个decorator，返回一个函数，所以，原来的now()函数仍然存在，只是现在同名的now变量指向了新的函数，于是调用now()将执行新函数

### 偏函数 partial func
在原先函数的基础上，穿入参数，形成新函数

In [29]:
import functools
int2 = functools.partial(int, base=2)
int2("1111")

15

# logging
用来打印错误信息

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

ZeroDivisionError: division by zero

# 单元测试
测试驱动开发TDD test drive development
可以在单元测试中编写两个特殊的setUp()和tearDown()方法。这两个方法会分别在每调用一个测试方法的前后分别被执行。

In [9]:
import unittest


class Dict(dict):

    def __init__(self, **kw):
        super().__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value


class TestDict(unittest.TestCase):

    def test_init(self):
        d = Dict(a=1, b='test')
        self.assertEqual(d.a, 1)
        self.assertEqual(d.b, 'test')
        self.assertTrue(isinstance(d, dict))

    def test_key(self):
        d = Dict()
        d['key'] = 'value'
        self.assertEqual(d.key, 'value')

    def test_attr(self):
        d = Dict()
        d.key = 'value'
        self.assertTrue('key' in d)
        self.assertEqual(d['key'], 'value')

    def test_keyerror(self):
        d = Dict()
        with self.assertRaises(KeyError):
            value = d['empty']

    def test_attrerror(self):
        d = Dict()
        with self.assertRaises(AttributeError):
            value = d.empty


if __name__ == '__main__':
    unittest.main()

E
ERROR: /Users/andrew/Library/Jupyter/runtime/kernel-9c78cb54-c387-45ac-9aea-ac97ac87e0da (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/andrew/Library/Jupyter/runtime/kernel-9c78cb54-c387-45ac-9aea-ac97ac87e0da'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/anaconda3/envs/test37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 文档测试
用文档的测试用例

In [6]:
def fact(n):
    '''
    Calculate 1*2*...*n

    >>> fact(1)
    1
    >>> fact(10)
    3628800
    >>> fact(-1)
    Traceback (most recent call last):
    ...
    ValueError
    '''
    if n < 1:
        raise ValueError()
    if n == 1:
        return 1
    return n * fact(n - 1)


if __name__ == '__main__':
    import doctest

    doctest.testmod()
